In [ ]:
"""
LINCS 2020数据加载和预处理脚本 - 优化版
修正DOS过滤和时间点匹配问题
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import cosine, pdist, squareform
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from numba import jit, prange
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import warnings
import gc
from tqdm import tqdm
warnings.filterwarnings('ignore')


class LINCS2020DataLoader:
    """
    加载和预处理LINCS 2020数据 - 优化版
    修正DOS过滤和时间点匹配
    """
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.compound_info = None
        self.inst_info = None  
        self.signatures = None
        self.decompressed_files = []
        
        print(f"\n{'='*80}")
        print(f"🔬 LINCS 2020 Data Loader - Optimized Version")
        print(f"{'='*80}")
        print(f"Data directory: {self.data_dir}")
        print(f"Dataset: Expanded CMap LINCS Resource 2020")
        print(f"{'='*80}\n")
    
    def load_gene_info(self):
        """加载基因信息，获取978个landmark genes"""
        gene_file = self.data_dir / "geneinfo_beta.txt"
        
        if not gene_file.exists():
            raise FileNotFoundError(
                f"❌ Gene info file not found: {gene_file}\n"
                f"   Please download 'geneinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"📖 Loading gene information...")
        print(f"   File: {gene_file.name}")
        
        gene_info = pd.read_csv(gene_file, sep='\t')
        
        print(f"   ✓ Loaded {len(gene_info):,} genes")
        print(f"   ✓ Columns: {list(gene_info.columns)}")
        
        if 'feature_space' in gene_info.columns:
            landmark_mask = gene_info['feature_space'] == 'landmark'
            landmark_genes = gene_info[landmark_mask].copy()
            print(f"   ✓ Using 'feature_space' column to identify landmarks")
        else:
            raise ValueError(
                f"Cannot identify landmark genes. 'feature_space' column not found.\n"
                f"Available columns: {list(gene_info.columns)}"
            )

        print(f"   ✓ Landmark genes: {len(landmark_genes):,}")
        print(f"   ✓ Expected: 978")

        self.landmark_col_indices = np.where(landmark_mask.values)[0]
        print(f"\n   Landmark column indices (first 10): {self.landmark_col_indices[:10]}")

        self.landmark_gene_ids = set(landmark_genes['gene_id'].astype(str).values)
        self.landmark_gene_symbols = set(landmark_genes['gene_symbol'].astype(str).values)

        print(f"   Sample IDs: {list(self.landmark_gene_ids)[:5]}")
        print(f"   Sample symbols: {list(self.landmark_gene_symbols)[:5]}")

        self.gene_info = gene_info
        return gene_info
    
    def load_cell_info(self):
        """加载细胞系信息"""
        cell_file = self.data_dir / "cellinfo_beta.txt"
        
        if not cell_file.exists():
            raise FileNotFoundError(
                f"❌ Cell info file not found: {cell_file}\n"
                f"   Please download 'cellinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"\n📖 Loading cell information...")
        print(f"   File: {cell_file.name}")
        
        cell_info = pd.read_csv(cell_file, sep='\t')
        
        print(f"   ✓ Loaded {len(cell_info):,} cell lines")
        print(f"   ✓ Columns: {list(cell_info.columns[:10])}...")
        
        if 'cell_iname' in cell_info.columns:
            print(f"   ✓ Unique cell lines (cell_iname): {cell_info['cell_iname'].nunique()}")
        
        print(f"\n   Sample cell lines:")
        for cell in cell_info['cell_iname'].head(5).values:
            print(f"     - {cell}")
        
        self.cell_info = cell_info
        return cell_info
    
    def load_compound_info(self):
        """加载化合物信息"""
        compound_file = self.data_dir / "compoundinfo_beta.txt"
        
        if not compound_file.exists():
            print(f"⚠️  Compound info file not found: {compound_file}")
            print(f"   This file is optional but recommended.")
            return None
        
        print(f"\n📖 Loading compound information...")
        print(f"   File: {compound_file.name}")
        
        compound_info = pd.read_csv(compound_file, sep='\t')
        
        print(f"   ✓ Loaded {len(compound_info):,} compounds")
        print(f"   ✓ Columns: {list(compound_info.columns)}")
        
        if 'pert_id' in compound_info.columns:
            print(f"   ✓ Unique perturbagens: {compound_info['pert_id'].nunique()}")
        
        print(f"\n   Sample compounds:")
        for _, row in compound_info.head(3).iterrows():
            name = row.get('cmap_name', 'Unknown')
            print(f"     - {row['pert_id']}: {name}")
        
        self.compound_info = compound_info
        return compound_info
    
    def load_instance_info(self):
        """加载实例信息"""
        inst_file = self.data_dir / "instinfo_beta.txt"

        if not inst_file.exists():
            raise FileNotFoundError(
                f"❌ Instance info file not found: {inst_file}\n"
                f"   Please download 'instinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )

        print(f"\n📖 Loading instance information...")
        print(f"   File: {inst_file.name}")

        inst_info = pd.read_csv(inst_file, sep='\t')

        print(f"   ✓ Loaded {len(inst_info):,} instances")
        print(f"   ✓ Columns: {list(inst_info.columns[:10])}...")

        if 'inst_id' in inst_info.columns:
            self.instance_join_col = 'inst_id'
        elif 'sample_id' in inst_info.columns:
            self.instance_join_col = 'sample_id'
        else:
            raise ValueError(
                f"Neither 'inst_id' nor 'sample_id' found in instinfo_beta.txt. "
                f"Available columns: {list(inst_info.columns)}"
            )

        print(f"   ✓ Using '{self.instance_join_col}' as join key")

        self.inst_info = inst_info
        return inst_info

    def decompress_gctx_file(self, gctx_file):
        """如果GCTX文件被压缩，则解压"""
        gctx_file = Path(gctx_file)
        
        if not gctx_file.exists():
            raise FileNotFoundError(f"GCTX file not found: {gctx_file}")
        
        if str(gctx_file).endswith('.gz'):
            print(f"⚠️  Detected compressed GCTX file: {gctx_file.name}")
            
            decompressed_dir = self.data_dir / "_decompressed"
            decompressed_dir.mkdir(exist_ok=True)
            
            output_path = decompressed_dir / gctx_file.stem
            
            if output_path.exists():
                print(f"✓ Found existing decompressed file: {output_path.name}")
                return str(output_path)
            
            print(f"📦 Decompressing...")
            with gzip.open(gctx_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            print(f"✓ Decompressed to: {output_path}")
            return str(output_path)
        
        return str(gctx_file)
    
    def read_gctx(self, gctx_file, use_landmark_only=True):
        """读取GCTX文件 - 内存优化版"""
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        file_size_gb = Path(gctx_file).stat().st_size / (1024**3)
        print(f"   File size: {file_size_gb:.2f} GB")
        
        if file_size_gb > 50:
            print(f"   ⚠️  Large file detected. Using memory-optimized loading...")
        
        gctx_file = self.decompress_gctx_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            print(f"📊 Inspecting HDF5 structure...")
            print(f"   Available keys: {list(f.keys())}")
            
            if '/0/DATA/0/matrix' in f:
                matrix_dataset = f['/0/DATA/0/matrix']
                row_path = '/0/META/ROW'
                col_path = '/0/META/COL'
            elif '/matrix' in f:
                matrix_dataset = f['/matrix']
                row_path = '/row'
                col_path = '/col'
            else:
                raise ValueError(f"Cannot find matrix in GCTX file. Available keys: {list(f.keys())}")
            
            matrix_shape = matrix_dataset.shape
            print(f"   ✓ Matrix shape: {matrix_shape} (samples × genes)")
            
            print(f"\n📋 Loading metadata...")
            
            # 读取样本元数据
            sample_meta = {}
            if row_path in f:
                print(f"   Reading sample metadata from: {row_path}")
                print(f"   Available row fields: {list(f[row_path].keys())}")
                
                for key in f[row_path].keys():
                    data = f[f'{row_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            sample_meta[key] = data.astype(str)
                    else:
                        sample_meta[key] = data
                
                print(f"   ✓ Loaded {len(sample_meta)} sample metadata fields")
            else:
                raise ValueError(f"Cannot find row metadata at: {row_path}")
            
            # 读取基因元数据
            gene_meta = {}
            if col_path in f:
                print(f"   Reading gene metadata from: {col_path}")
                print(f"   Available col fields: {list(f[col_path].keys())}")
                
                for key in f[col_path].keys():
                    data = f[f'{col_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            gene_meta[key] = data.astype(str)
                    else:
                        gene_meta[key] = data
                
                print(f"   ✓ Loaded {len(gene_meta)} gene metadata fields")
            else:
                raise ValueError(f"Cannot find col metadata at: {col_path}")
            
            sample_df = pd.DataFrame(sample_meta)
            gene_df = pd.DataFrame(gene_meta)

            # 自动检测并交换ROW/COL
            if self.gene_info is not None:
                n_features_expected = len(self.gene_info)
                if len(sample_df) == n_features_expected and len(gene_df) != n_features_expected:
                    print(f"\n   ⚠️ Detected ROW/COL swap. Correcting...")
                    sample_df, gene_df = gene_df, sample_df

            print(f"\n   Sample metadata columns: {list(sample_df.columns[:10])}...")
            print(f"   Gene metadata columns: {list(gene_df.columns)}")
            
            # 确定landmark基因的列索引
            landmark_col_indices = None
            if use_landmark_only:
                print(f"\n🔬 Filtering to landmark genes...")

                if (self.gene_info is None) or (not hasattr(self, "landmark_col_indices")):
                    print(f"   Loading gene info to get landmark indices...")
                    self.load_gene_info()

                if len(self.gene_info) != matrix_shape[1]:
                    print(f"   ⚠️  Warning: geneinfo rows ({len(self.gene_info)}) "
                        f"!= GCTX feature count ({matrix_shape[1]})")
                
                landmark_col_indices = np.array(self.landmark_col_indices, dtype=int)
                print(f"   ✓ Using {len(landmark_col_indices)} landmark features "
                    f"out of {matrix_shape[1]} total")
            
            # 内存优化：分块读取
            print(f"\n🎯 Loading data (memory-optimized)...")
            if landmark_col_indices is not None:
                print(f"   Reading {len(landmark_col_indices)} columns out of {matrix_shape[1]}...")
                chunk_size = 610000
                chunks = []
                
                for start_idx in range(0, matrix_shape[0], chunk_size):
                    end_idx = min(start_idx + chunk_size, matrix_shape[0])
                    print(f"   Loading rows {start_idx:,} to {end_idx:,}... ({end_idx/matrix_shape[0]*100:.1f}%)", end='\r')
                    
                    chunk = matrix_dataset[start_idx:end_idx, landmark_col_indices].astype(np.float32)
                    chunks.append(chunk)
                    
                    if len(chunks) >= 10:
                        print(f"\n   Consolidating chunks...")
                        merged = np.vstack(chunks)
                        chunks = [merged]
                        gc.collect()
                
                print(f"\n   Finalizing matrix...")
                matrix = np.vstack(chunks) if len(chunks) > 1 else chunks[0]
                del chunks
                gc.collect()
            else:
                print(f"   Reading full matrix...")
                matrix = matrix_dataset[:].astype(np.float32)
            
            print(f"   ✓ Final matrix shape: {matrix.shape}")
            print(f"   ✓ Memory usage: {matrix.nbytes / (1024**3):.2f} GB")
            print(f"   ✓ Data type: {matrix.dtype}")
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self):
        """加载Level 4数据"""
        level4_file = self.data_dir / "level4_beta_trt_cp_n1805898x12328.gctx"
        
        if not level4_file.exists():
            pattern = self.data_dir / "level4_beta_trt_cp*.gctx"
            files = glob.glob(str(pattern))
            
            if not files:
                raise FileNotFoundError(
                    f"❌ Level 4 file not found: {level4_file}\n"
                    f"   Please download from: https://clue.io/data/CMap2020#LINCS2020"
                )
            
            level4_file = files[0]
        
        print(f"\n{'='*80}")
        print(f"📖 Loading Level 4 Signatures")
        print(f"{'='*80}")
        print(f"File: {level4_file.name}")
        
        matrix, sample_meta, gene_meta = self.read_gctx(
            level4_file, 
            use_landmark_only=True
        )

        # 用instinfo补齐metadata
        if 'id' in sample_meta.columns:
            sample_meta = sample_meta.rename(columns={'id': 'sample_id'})
        elif 'sample_id' not in sample_meta.columns:
            raise ValueError("Cannot find 'id' or 'sample_id' in GCTX ROW metadata")

        if self.inst_info is None:
            self.load_instance_info()

        inst_info = self.inst_info
        join_col = getattr(self, "instance_join_col", None)
        
        if join_col is None:
            join_col = 'sample_id' if 'sample_id' in inst_info.columns else 'inst_id'

        if join_col not in sample_meta.columns:
            if join_col == 'inst_id' and 'sample_id' in sample_meta.columns:
                print("   ⚠️ Using 'sample_id' instead of 'inst_id'")
                join_col = 'sample_id'
            else:
                raise ValueError(
                    f"Join column '{join_col}' not found in GCTX metadata. "
                    f"Available: {list(sample_meta.columns)}"
                )

        print(f"   🔗 Merging GCTX metadata with instinfo on '{join_col}'...")
        merged_meta = sample_meta.merge(inst_info, on=join_col, how='left')

        if 'pert_id' not in merged_meta.columns:
            raise ValueError("After merging, 'pert_id' is still missing")

        n_missing = merged_meta['pert_id'].isna().sum()
        if n_missing > 0:
            print(f"   ⚠️ {n_missing:,} rows have missing pert_id")

        self.signatures = {
            'matrix': matrix,
            'row_meta': merged_meta,
            'col_meta': gene_meta
        }

        return matrix, merged_meta, gene_meta
    
    def calculate_cosine_similarity_to_nearest_replicate(
        self, 
        matrix: np.ndarray, 
        pert_ids: pd.Series
    ) -> np.ndarray:
        """
        使用Numba JIT编译的超高效版本
        
        计算每个样本与其最近同化合物复制品的余弦相似度
        """
       
        print(f"\n📊 Calculating similarities (Numba-accelerated)...")
        
        # 预归一化
        matrix_norm = normalize(matrix, norm='l2', axis=1).astype(np.float32)
        
        @jit(nopython=True, parallel=True, fastmath=True)
        def compute_max_similarities(data, indices, n_total):
            """Numba加速的核心计算"""
            n = len(indices)
            result = np.full(n_total, -np.inf, dtype=np.float32)
            
            for i in prange(n):
                idx_i = indices[i]
                vec_i = data[i]
                max_sim = -np.inf
                
                for j in range(n):
                    if i != j:
                        # 点积（已归一化 = 余弦相似度）
                        sim = np.dot(vec_i, data[j])
                        if sim > max_sim:
                            max_sim = sim
                
                result[idx_i] = max_sim
            
            return result
        
        nearest_similarities = np.zeros(len(pert_ids), dtype=np.float32)
        
        # 按化合物分组
        pert_ids_array = pert_ids.values
        unique_perts = np.unique(pert_ids_array)
        
        from tqdm import tqdm
        for pert_id in tqdm(unique_perts, desc="   Computing"):
            mask = pert_ids_array == pert_id
            indices = np.where(mask)[0]
            
            if len(indices) < 2:
                nearest_similarities[indices] = 0.0
                continue
            
            pert_data = matrix_norm[mask]
            sims = compute_max_similarities(pert_data, indices, len(pert_ids))
            nearest_similarities[indices] = sims[indices]
        
        print(f"   ✓ Mean similarity: {nearest_similarities.mean():.4f}")
        return nearest_similarities
    
    def prepare_training_data(
        self, 
        min_observations_per_compound=5,
        min_replicate_similarity=0.12,
        dose_range=(1.0, 20.0),
        valid_timepoints=[6, 24],  # 修改为数值列表
        min_cell_lines=5,
        max_cell_lines=40,
        remove_dos=True
    ):
        """
        准备训练数据
        修正DOS过滤和时间点匹配
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta'].copy()
        col_meta = self.signatures['col_meta']
        
        print(f"\n[DEBUG] row_meta shape: {row_meta.shape}")
        print(f"[DEBUG] columns (first 15): {list(row_meta.columns[:15])}")
        print(f"[DEBUG] Example row:")
        print(row_meta.iloc[0][['sample_id', 'pert_id', 'pert_type', 'cell_iname', 'pert_time', 'pert_dose']])

        print("\n" + "=" * 80)
        print("🔍 DRUGREFLECTOR QUALITY CONTROL PIPELINE - Optimized")
        print("=" * 80)
        print(f"Initial samples: {len(row_meta):,}")
        print(f"Initial memory: {matrix.nbytes / (1024**3):.2f} GB")
        
        print(f"\n📋 Available metadata columns:")
        for col in row_meta.columns[:15]:
            sample_val = row_meta[col].iloc[0] if len(row_meta) > 0 else 'N/A'
            print(f"   - {col}: {sample_val}")
        if len(row_meta.columns) > 15:
            print(f"   ... and {len(row_meta.columns) - 15} more")
        
        # 检查必需字段
        required_fields = ['pert_id']
        missing_fields = [f for f in required_fields if f not in row_meta.columns]
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
        
        # 确定细胞系ID字段
        cell_id_col = None
        for possible_col in ['cell_id', 'cell_iname', 'cell_mfc_name']:
            if possible_col in row_meta.columns:
                cell_id_col = possible_col
                print(f"\n✓ Using '{cell_id_col}' as cell line identifier")
                break
        
        if cell_id_col is None:
            print(f"\n⚠️  Warning: Cannot find cell line identifier")
        
        valid_mask = np.ones(len(row_meta), dtype=bool)
        
        initial_compounds = row_meta['pert_id'].nunique()
        print(f"\nInitial compounds: {initial_compounds:,}")
        
        # ========== Filter 1: Remove DOS compounds (优化) ==========
        if remove_dos:
            print(f"\n{'='*80}")
            print(f"FILTER 1: Remove DOS compounds (keep trt_cp only)")
            print(f"{'='*80}")
            
            # 使用pert_type字段，只保留trt_cp
            if 'pert_type' in row_meta.columns:
                # 检查pert_type的值分布
                print(f"   pert_type value counts:")
                pert_type_counts = row_meta['pert_type'].value_counts()
                for ptype, count in pert_type_counts.items():
                    print(f"     - {ptype}: {count:,} samples")
                
                # 只保留trt_cp类型
                dos_mask = row_meta['pert_type'] == 'trt_cp'
                n_removed = (~dos_mask).sum()
                
                print(f"\n  ✓ Keeping only 'trt_cp' perturbations")
                print(f"  Removed {n_removed:,} non-trt_cp observations")
            else:
                # 降级方案：如果没有pert_type，使用更保守的BRD-K规则
                print(f"   ⚠️ 'pert_type' not found, using fallback method")
                # 只过滤明确标记为DOS的
                dos_mask = ~row_meta['pert_id'].str.contains('DOS', case=False, na=False)
                n_removed = (~dos_mask).sum()
                print(f"  Removed {n_removed:,} DOS observations")
            
            valid_mask &= dos_mask
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        # ========== Filter 2: Minimum observations ==========
        print(f"\n{'='*80}")
        print(f"FILTER 2: Remove compounds with <{min_observations_per_compound} observations")
        print(f"{'='*80}")
        
        valid_meta = row_meta[valid_mask]
        obs_counts = valid_meta.groupby('pert_id').size()
        valid_perts = obs_counts[obs_counts >= min_observations_per_compound].index
        
        print(f"  Compounds with ≥{min_observations_per_compound} observations: "
              f"{len(valid_perts):,}/{obs_counts.nunique():,}")
        
        obs_mask = row_meta['pert_id'].isin(valid_perts)
        valid_mask &= obs_mask
        
        print(f"  Remaining samples: {valid_mask.sum():,}")
        print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        del valid_meta, obs_counts, obs_mask
        gc.collect()
            
        # ========== Filter 3: Cosine similarity ==========
        print(f"\n{'='*80}")
        print(f"FILTER 3: Remove observations with cosine similarity <{min_replicate_similarity}")
        print(f"{'='*80}")
        
        valid_indices = np.where(valid_mask)[0]
        valid_matrix = matrix[valid_mask]
        valid_pert_ids = row_meta.loc[valid_mask, 'pert_id'].reset_index(drop=True)
        
        nearest_similarities = self.calculate_cosine_similarity_to_nearest_replicate(
            valid_matrix, 
            valid_pert_ids
        )
        
        full_similarities = np.zeros(len(row_meta), dtype=np.float32)
        full_similarities[valid_indices] = nearest_similarities
        
        sim_mask = (full_similarities >= min_replicate_similarity) | (~valid_mask)
        n_removed_sim = (~sim_mask & valid_mask).sum()
        valid_mask &= sim_mask
        
        print(f"  Removed {n_removed_sim:,} low-similarity observations")
        print(f"  Remaining samples: {valid_mask.sum():,}")
        
        del valid_matrix, valid_pert_ids, nearest_similarities, full_similarities, sim_mask
        gc.collect()
        
        # ========== Filter 4: Dose selection (修正版 + 详细统计) ==========
        print(f"\n{'='*80}")
        print(f"FILTER 4: Select most frequent dose in range {dose_range[0]}-{dose_range[1]} µM")
        print(f"{'='*80}")

        if 'pert_dose' in row_meta.columns:
            
            # Step 1: 解析剂量
            print(f"  Step 1/5: Parsing dose values...")
            dose_str = row_meta['pert_dose'].astype(str)
            row_meta['dose_value'] = pd.to_numeric(
                dose_str.str.extract(r'([\d.]+)', expand=False), 
                errors='coerce'
            )
            row_meta['dose_unit'] = dose_str.str.extract(r'([a-zA-Z]+)', expand=False).str.lower().fillna('um')
            
            n_parsed = (~pd.isna(row_meta['dose_value'])).sum()
            n_missing = pd.isna(row_meta['dose_value']).sum()
            print(f"  ✓ Successfully parsed: {n_parsed:,} entries")
            print(f"  ⚠️  Missing/invalid doses: {n_missing:,} entries")
            
            # Step 2: 转换单位
            print(f"\n  Step 2/5: Converting to µM...")
            dose_value = row_meta['dose_value'].values
            dose_unit = row_meta['dose_unit'].values
            
            conditions = [
                dose_unit == 'nm',
                dose_unit == 'mm'
            ]
            choices = [
                dose_value / 1000,
                dose_value * 1000
            ]
            row_meta['dose_uM'] = np.select(conditions, choices, default=dose_value)
            
            # 统计单位分布
            print(f"  Dose unit distribution:")
            unit_counts = row_meta['dose_unit'].value_counts()
            for unit, count in unit_counts.head(5).items():
                print(f"    - {unit}: {count:,} samples")
            
            # Step 3: 剂量分布统计 (在过滤前)
            print(f"\n  Step 3/5: Dose distribution analysis...")
            
            # 只分析有效剂量值
            valid_doses = row_meta.loc[valid_mask & (~pd.isna(row_meta['dose_uM'])), 'dose_uM']
            
            if len(valid_doses) > 0:
                print(f"\n  📊 Dose statistics (before filtering):")
                print(f"    Total samples with valid dose: {len(valid_doses):,}")
                print(f"    Dose range: {valid_doses.min():.4f} - {valid_doses.max():.2f} µM")
                print(f"    Mean dose: {valid_doses.mean():.4f} µM")
                print(f"    Median dose: {valid_doses.median():.4f} µM")
                
                # 剂量分布（按区间）
                dose_bins = [0, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 500, np.inf]
                dose_labels = ['<0.1', '0.1-0.5', '0.5-1', '1-2', '2-5', '5-10', 
                            '10-20', '20-50', '50-100', '100-500', '>500']
                dose_binned = pd.cut(valid_doses, bins=dose_bins, labels=dose_labels, include_lowest=True)
                dose_dist = dose_binned.value_counts().sort_index()
                
                print(f"\n  Dose distribution by range:")
                for dose_range_label, count in dose_dist.items():
                    pct = count / len(valid_doses) * 100
                    print(f"    {dose_range_label:>10} µM: {count:>8,} ({pct:>5.1f}%)")
                
                # 统计最常见的精确剂量值
                print(f"\n  Top 20 most frequent exact doses:")
                top_doses = valid_doses.value_counts().head(20)
                for dose_val, count in top_doses.items():
                    pct = count / len(valid_doses) * 100
                    print(f"    {dose_val:>8.4f} µM: {count:>8,} samples ({pct:>5.2f}%)")
            
            # Step 4: 应用剂量范围过滤
            print(f"\n  Step 4/5: Filtering dose range {dose_range[0]}-{dose_range[1]} µM...")
            dose_range_mask = (
                (row_meta['dose_uM'] >= dose_range[0]) & 
                (row_meta['dose_uM'] <= dose_range[1]) &
                (~pd.isna(row_meta['dose_uM']))
            )
            
            n_in_range = (valid_mask & dose_range_mask).sum()
            n_out_range = (valid_mask & ~dose_range_mask).sum()
            print(f"  ✓ Samples in target range: {n_in_range:,}")
            print(f"  ✓ Samples outside range: {n_out_range:,}")
            
            # 先应用dose_range_mask
            n_before_dose = valid_mask.sum()
            valid_mask &= dose_range_mask
            n_removed_range = n_before_dose - valid_mask.sum()
            print(f"  ✓ Removed {n_removed_range:,} samples outside dose range")
            
            # Step 5: 在已过滤的数据中找最常见剂量
            print(f"\n  Step 5/5: Finding most frequent dose per compound...")
            valid_meta = row_meta[valid_mask].copy()
            unique_compounds = valid_meta['pert_id'].nunique()
            print(f"  Processing {unique_compounds:,} compounds...")
            
            tqdm.pandas(desc="    Finding modal doses")
            most_common_doses = (
                valid_meta.groupby('pert_id')['dose_uM']
                .progress_apply(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
            )
            
            print(f"  ✓ Identified modal doses for {len(most_common_doses):,} compounds")
            
            # 统计modal dose分布
            print(f"\n  📊 Modal dose distribution:")
            modal_dose_counts = most_common_doses.value_counts().head(15)
            for dose_val, count in modal_dose_counts.items():
                pct = count / len(most_common_doses) * 100
                print(f"    {dose_val:>8.4f} µM: {count:>5,} compounds ({pct:>5.2f}%)")
            
            # 映射到所有行
            row_meta['most_common_dose'] = row_meta['pert_id'].map(most_common_doses)
            
            # 只在已经通过valid_mask的样本中筛选modal dose
            dose_modal_mask = (
                (row_meta['dose_uM'] == row_meta['most_common_dose']) &
                (~pd.isna(row_meta['most_common_dose']))
            )
            
            n_before_modal = valid_mask.sum()
            valid_mask &= dose_modal_mask
            n_removed_modal = n_before_modal - valid_mask.sum()
            
            print(f"\n  ✓ Removed {n_removed_modal:,} samples with non-modal dose")
            print(f"  ✓ Remaining samples: {valid_mask.sum():,}")
            print(f"  ✓ Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            
            # ========== 可视化部分 ==========
            print(f"\n  📊 Generating dose distribution visualizations...")
            
            # 创建输出目录
            viz_dir = Path("E:/科研/Models/drugreflector/visualizations")
            viz_dir.mkdir(exist_ok=True, parents=True)
            
            # 图1: 剂量分布直方图（对数刻度）
            plt.figure(figsize=(14, 10))
            
            # 子图1: 过滤前的剂量分布
            plt.subplot(2, 2, 1)
            valid_doses_before = row_meta.loc[
                row_meta.index[valid_indices] if 'valid_indices' in locals() else slice(None), 
                'dose_uM'
            ].dropna()
            
            if len(valid_doses_before) > 0:
                # 使用对数bins
                log_bins = np.logspace(np.log10(0.001), np.log10(1000), 50)
                plt.hist(valid_doses_before, bins=log_bins, edgecolor='black', alpha=0.7, color='steelblue')
                plt.axvline(dose_range[0], color='red', linestyle='--', linewidth=2, label=f'Range: {dose_range[0]}-{dose_range[1]} µM')
                plt.axvline(dose_range[1], color='red', linestyle='--', linewidth=2)
                plt.xscale('log')
                plt.xlabel('Dose (µM, log scale)', fontsize=11)
                plt.ylabel('Number of samples', fontsize=11)
                plt.title('Dose Distribution (Before Filtering)', fontsize=13, fontweight='bold')
                plt.legend()
                plt.grid(True, alpha=0.3)
            
            # 子图2: 过滤后的剂量分布
            plt.subplot(2, 2, 2)
            final_doses = row_meta.loc[valid_mask, 'dose_uM'].dropna()
            
            if len(final_doses) > 0:
                log_bins = np.logspace(np.log10(dose_range[0]), np.log10(dose_range[1]), 30)
                plt.hist(final_doses, bins=log_bins, edgecolor='black', alpha=0.7, color='forestgreen')
                plt.xscale('log')
                plt.xlabel('Dose (µM, log scale)', fontsize=11)
                plt.ylabel('Number of samples', fontsize=11)
                plt.title('Dose Distribution (After Filtering)', fontsize=13, fontweight='bold')
                plt.grid(True, alpha=0.3)
            
            # 子图3: Top 30 最常见的精确剂量值（柱状图）
            plt.subplot(2, 2, 3)
            top_doses_final = final_doses.value_counts().head(30)
            
            if len(top_doses_final) > 0:
                bars = plt.barh(range(len(top_doses_final)), top_doses_final.values, color='coral', edgecolor='black')
                plt.yticks(range(len(top_doses_final)), 
                        [f'{d:.4f} µM' for d in top_doses_final.index])
                plt.xlabel('Number of samples', fontsize=11)
                plt.ylabel('Dose (µM)', fontsize=11)
                plt.title('Top 30 Most Frequent Exact Doses', fontsize=13, fontweight='bold')
                plt.grid(axis='x', alpha=0.3)
                
                # 添加数值标签
                for i, (bar, val) in enumerate(zip(bars, top_doses_final.values)):
                    plt.text(val, i, f' {val:,}', va='center', fontsize=8)
            
            # 子图4: Modal dose分布（化合物层面）
            plt.subplot(2, 2, 4)
            modal_doses_final = most_common_doses.value_counts().head(20)
            
            if len(modal_doses_final) > 0:
                bars = plt.barh(range(len(modal_doses_final)), modal_doses_final.values, 
                            color='mediumpurple', edgecolor='black')
                plt.yticks(range(len(modal_doses_final)), 
                        [f'{d:.4f} µM' for d in modal_doses_final.index])
                plt.xlabel('Number of compounds', fontsize=11)
                plt.ylabel('Modal Dose (µM)', fontsize=11)
                plt.title('Top 20 Modal Doses (Per Compound)', fontsize=13, fontweight='bold')
                plt.grid(axis='x', alpha=0.3)
                
                # 添加数值标签
                for i, (bar, val) in enumerate(zip(bars, modal_doses_final.values)):
                    plt.text(val, i, f' {val:,}', va='center', fontsize=8)
            
            plt.tight_layout()
            output_path = viz_dir / "dose_distribution_analysis.png"
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f"  ✓ Saved figure: {output_path}")
            plt.close()
            
            # 图2: 剂量范围分布（分区间统计）
            plt.figure(figsize=(12, 6))
            
            dose_bins = [0, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 500, np.inf]
            dose_labels = ['<0.1', '0.1-0.5', '0.5-1', '1-2', '2-5', '5-10', 
                        '10-20', '20-50', '50-100', '100-500', '>500']
            
            # 过滤前
            doses_before = row_meta.loc[
                row_meta.index[valid_indices] if 'valid_indices' in locals() else slice(None), 
                'dose_uM'
            ].dropna()
            binned_before = pd.cut(doses_before, bins=dose_bins, labels=dose_labels, include_lowest=True)
            dist_before = binned_before.value_counts().sort_index()
            
            # 过滤后
            doses_after = row_meta.loc[valid_mask, 'dose_uM'].dropna()
            binned_after = pd.cut(doses_after, bins=dose_bins, labels=dose_labels, include_lowest=True)
            dist_after = binned_after.value_counts().sort_index()
            
            x = np.arange(len(dose_labels))
            width = 0.35
            
            plt.bar(x - width/2, dist_before.values, width, label='Before filtering', 
                    alpha=0.8, color='steelblue', edgecolor='black')
            plt.bar(x + width/2, dist_after.values, width, label='After filtering', 
                    alpha=0.8, color='forestgreen', edgecolor='black')
            
            plt.xlabel('Dose Range (µM)', fontsize=12, fontweight='bold')
            plt.ylabel('Number of Samples', fontsize=12, fontweight='bold')
            plt.title('Sample Distribution by Dose Range (Before vs After Filtering)', 
                    fontsize=14, fontweight='bold')
            plt.xticks(x, dose_labels, rotation=45, ha='right')
            plt.legend(fontsize=11)
            plt.grid(axis='y', alpha=0.3)
            
            # 添加数值标签
            for i, (v_before, v_after) in enumerate(zip(dist_before.values, dist_after.values)):
                plt.text(i - width/2, v_before, f'{v_before:,}', ha='center', va='bottom', fontsize=8)
                plt.text(i + width/2, v_after, f'{v_after:,}', ha='center', va='bottom', fontsize=8)
            
            plt.tight_layout()
            output_path2 = viz_dir / "dose_range_comparison.png"
            plt.savefig(output_path2, dpi=300, bbox_inches='tight')
            print(f"  ✓ Saved figure: {output_path2}")
            plt.close()
            
            print(f"\n  ✅ Visualization complete!")
            print(f"     Figures saved to: {viz_dir}")
            
            # 清理临时列
            row_meta.drop(['dose_value', 'dose_unit', 'most_common_dose'], 
                        axis=1, inplace=True, errors='ignore')
            
            del valid_meta, dose_range_mask, dose_modal_mask, most_common_doses
            gc.collect()
        else:
            print(f"  ⚠️  'pert_dose' not found, skipping dose filter")
        
        # ========== Filter 5: Timepoint selection (优化) ==========
        print(f"\n{'='*80}")
        print(f"FILTER 5: Keep only measurements at {valid_timepoints} hours")
        print(f"{'='*80}")
        
        if 'pert_time' in row_meta.columns:
            # 转换为数值类型
            row_meta['time_numeric'] = pd.to_numeric(row_meta['pert_time'], errors='coerce')
            
            # 显示时间点分布
            print(f"   Available timepoints (numeric):")
            time_counts = row_meta['time_numeric'].value_counts().head(10)
            for time_val, count in time_counts.items():
                print(f"     - {time_val} hours: {count:,} samples")
            
            # 匹配valid_timepoints中的数值
            time_mask = row_meta['time_numeric'].isin(valid_timepoints)
            
            if time_mask.sum() == 0:
                print(f"  ⚠️  No samples match timepoints {valid_timepoints}")
                print(f"  Skipping timepoint filter...")
            else:
                n_before = valid_mask.sum()
                valid_mask &= time_mask
                n_removed = n_before - valid_mask.sum()
                
                print(f"  ✓ Kept samples at {valid_timepoints} hours")
                print(f"  Removed {n_removed:,} observations (invalid timepoint)")
                print(f"  Remaining samples: {valid_mask.sum():,}")
            
            del time_mask
            gc.collect()
        else:
            print(f"  ⚠️  'pert_time' not found, skipping timepoint filter")
        
        # ========== Filter 6: Cell line count ==========
        print(f"\n{'='*80}")
        print(f"FILTER 6: Remove compounds in <{min_cell_lines} or >{max_cell_lines} cell lines")
        print(f"{'='*80}")
        
        if cell_id_col is not None:
            valid_meta = row_meta[valid_mask]
            cell_line_counts = valid_meta.groupby('pert_id')[cell_id_col].nunique()
            valid_perts_cell = cell_line_counts[
                (cell_line_counts >= min_cell_lines) & 
                (cell_line_counts <= max_cell_lines)
            ].index
            
            print(f"  Compounds in {min_cell_lines}-{max_cell_lines} cell lines: "
                  f"{len(valid_perts_cell):,}/{len(cell_line_counts):,}")
            
            cell_mask = row_meta['pert_id'].isin(valid_perts_cell)
            n_before = valid_mask.sum()
            valid_mask &= cell_mask
            n_removed = n_before - valid_mask.sum()
            
            print(f"  Removed {n_removed:,} observations")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            
            del valid_meta, cell_line_counts, cell_mask
            gc.collect()
        else:
            print(f"  ⚠️  Cell line column not found, skipping")
        
        # ========== 最终提取数据 ==========
        print(f"\n{'='*80}")
        print(f"✅ FINAL DATASET")
        print(f"{'='*80}")
        
        final_matrix = matrix[valid_mask].copy()
        final_meta = row_meta[valid_mask].reset_index(drop=True)
        
        del matrix
        gc.collect()
        
        print(f"  Extracted {len(final_matrix):,} samples")
        print(f"  Memory usage: {final_matrix.nbytes / (1024**3):.2f} GB")
        
        # 创建标签
        unique_perts = sorted(final_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in final_meta['pert_id']], dtype=np.int32)
        
        final_compounds = len(unique_perts)
        final_samples = len(final_matrix)
        
        if cell_id_col:
            final_cells = final_meta[cell_id_col].nunique()
        else:
            final_cells = 'Unknown'
        
        print(f"\n  Total samples: {final_samples:,}")
        print(f"  Total compounds: {final_compounds:,}")
        print(f"  Cell lines: {final_cells}")
        print(f"  Gene features: {final_matrix.shape[1]}")
        
        compound_obs = final_meta.groupby('pert_id').size()
        print(f"  Samples per compound (mean): {final_samples / final_compounds:.1f}")
        print(f"  Samples per compound (median): {compound_obs.median():.0f}")
        
        n_compounds_100plus = (compound_obs > 100).sum()
        print(f"  Compounds with >100 observations: {n_compounds_100plus:,}")
        
        print(f"\n📊 Comparison with paper (SI page 2):")
        print(f"  Paper: 425,242 obs, 9,597 compounds, 52 cell lines")
        print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cell lines")
        
        if initial_compounds > 0:
            retention_rate = (final_compounds / initial_compounds) * 100
            print(f"  Compound retention rate: {retention_rate:.1f}%")
        
        # 基因名
        if (self.gene_info is not None) and hasattr(self, "landmark_col_indices"):
            gi = self.gene_info
            landmark_mask = gi['feature_space'] == 'landmark'
            landmark_geneinfo = gi[landmark_mask]
            gene_names = list(landmark_geneinfo['gene_symbol'].values)
            print(f"  ✓ Using geneinfo_beta.txt for gene names")
        else:
            gene_name_col = None
            for possible_col in ['gene_symbol', 'pr_gene_symbol', 'symbol', 'id']:
                if possible_col in col_meta.columns:
                    gene_name_col = possible_col
                    break
            if gene_name_col is None:
                gene_name_col = col_meta.columns[0]
            gene_names = list(col_meta[gene_name_col].values)
            print(f"  ⚠️ Using '{gene_name_col}' from col_meta for gene names")

        training_data = {
            'X': final_matrix,
            'y': labels,
            'folds': np.zeros(len(final_matrix), dtype=np.int32),
            'sample_meta': final_meta,
            'metadata': final_meta,
            'gene_names': gene_names,
            'compound_names': list(unique_perts),
            'pert_to_idx': pert_to_idx
        }

        gc.collect()
        
        return training_data
    
    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*80}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*80}")
        
        folds = np.zeros(len(sample_meta), dtype=np.int32)
        
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


def main():
    """主程序 - 优化版"""
    print("=" * 80)
    print("🧬 DRUGREFLECTOR DATA PREPROCESSING - Optimized Version")
    print("   Fixed DOS filtering and timepoint matching")
    print("=" * 80)
    
    data_dir = "E:/科研/Models/drugreflector/datasets/LINCS2020"
    loader = LINCS2020DataLoader(data_dir)
    
    try:
        # Step 1: 加载元数据
        print("\n" + "=" * 80)
        print("STEP 1: Loading metadata")
        print("=" * 80)
        gene_info = loader.load_gene_info()
        cell_info = loader.load_cell_info()
        compound_info = loader.load_compound_info()
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 80)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 80)
        matrix, row_meta, col_meta = loader.load_level4_signatures()
        
        # Step 3: 准备训练数据
        print("\n" + "=" * 80)
        print("STEP 3: Preparing training data")
        print("=" * 80)
        training_data = loader.prepare_training_data(
            min_observations_per_compound=5,
            min_replicate_similarity=0.12,
            dose_range=(1.0, 20.0),
            valid_timepoints=[6, 24],  # 修改为数值列表
            min_cell_lines=5,
            max_cell_lines=40,
            remove_dos=True
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 80)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 80)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存
        print("\n" + "=" * 80)
        print("STEP 5: Saving processed data")
        print("=" * 80)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True, parents=True)
        
        output_file = output_dir / "training_data_lincs2020_optimized_1121.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f, protocol=4)
        
        print(f"✓ Saved successfully!")
        print(f"   File size: {output_file.stat().st_size / (1024**2):.1f} MB")
        
        # 最终摘要
        print("\n" + "=" * 80)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 80)
        print(f"📁 Output: {output_file}")
        print(f"\n📊 Final dataset:")
        print(f"   • Samples: {len(training_data['X']):,}")
        print(f"   • Compounds: {len(training_data['compound_names']):,}")
        print(f"   • Genes: {training_data['X'].shape[1]}")
        print(f"   • Memory: {training_data['X'].nbytes / (1024**3):.2f} GB")
        
        # 与论文对比
        paper_samples = 425242
        paper_compounds = 9597
        our_samples = len(training_data['X'])
        our_compounds = len(training_data['compound_names'])
        
        print(f"\n📈 Comparison with paper:")
        print(f"   Samples: {our_samples:,} / {paper_samples:,} ({our_samples/paper_samples*100:.1f}%)")
        print(f"   Compounds: {our_compounds:,} / {paper_compounds:,} ({our_compounds/paper_compounds*100:.1f}%)")
        
        print(f"\n🎯 Ready for training!")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 80)
        print("❌ ERROR")
        print("=" * 80)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        return None


if __name__ == "__main__":
    training_data = main()

🧬 DRUGREFLECTOR DATA PREPROCESSING - Optimized Version
   Fixed DOS filtering and timepoint matching

🔬 LINCS 2020 Data Loader - Optimized Version
Data directory: E:\科研\Models\drugreflector\datasets\LINCS2020
Dataset: Expanded CMap LINCS Resource 2020


STEP 1: Loading metadata
📖 Loading gene information...
   File: geneinfo_beta.txt
   ✓ Loaded 12,328 genes
   ✓ Columns: ['gene_id', 'gene_symbol', 'ensembl_id', 'gene_title', 'gene_type', 'src', 'feature_space']
   ✓ Using 'feature_space' column to identify landmarks
   ✓ Landmark genes: 978
   ✓ Expected: 978

   Landmark column indices (first 10): [2154 2155 2156 2157 2158 2159 2160 2161 2162 2163]
   Sample IDs: ['10320', '2523', '10523', '11182', '596']
   Sample symbols: ['ALDH7A1', 'PRKACA', 'PROS1', 'TRAPPC6A', 'RFX5']

📖 Loading cell information...
   File: cellinfo_beta.txt
   ✓ Loaded 240 cell lines
   ✓ Columns: ['cell_iname', 'cellosaurus_id', 'donor_age', 'donor_age_death', 'donor_disease_age_onset', 'doubling_time', 'grow

   Computing: 100%|██████████| 22731/22731 [17:17<00:00, 21.91it/s]  


   ✓ Mean similarity: 0.3843
  Removed 37,549 low-similarity observations
  Remaining samples: 1,739,580

FILTER 4: Remove compounds in <5 or >40 cell lines
  Compounds in 5-40 cell lines: 10,733/22,528
  Removed 544,409 observations
  Remaining samples: 1,195,171
  Remaining compounds: 10,733

FILTER 5: Select most frequent dose in range 1.0-20.0 µM
  Step 1/4: Parsing dose values...
  ✓ Parsed 1,805,898 dose entries
  Step 2/4: Converting to µM...
  ✓ Converted doses to µM
  Step 3/4: Filtering dose range 1.0-20.0 µM...
  ✓ Samples in valid dose range (after previous filters): 780,076
  ✓ Removed 415,095 samples outside dose range
  Step 4/4: Finding most frequent dose per compound...
  Processing 10,721 compounds...


    Finding modal doses: 100%|██████████| 10721/10721 [00:02<00:00, 5339.39it/s]


  ✓ Identified modal doses for 10,721 compounds
  ✓ Removed 298,932 samples with non-modal dose
  ✓ Remaining samples: 481,144
  ✓ Remaining compounds: 10,721

FILTER 6: Keep only measurements at [6, 24] hours
   Available timepoints (numeric):
     - 24.0 hours: 1,338,831 samples
     - 6.0 hours: 398,943 samples
     - 3.0 hours: 31,925 samples
     - 48.0 hours: 18,614 samples
     - 4.0 hours: 11,372 samples
     - 72.0 hours: 2,308 samples
     - 2.0 hours: 1,135 samples
     - 12.0 hours: 1,083 samples
     - -666.0 hours: 581 samples
     - 120.0 hours: 367 samples
  ✓ Kept samples at [6, 24] hours
  Removed 7,046 observations (invalid timepoint)
  Remaining samples: 474,098

✅ FINAL DATASET
  Extracted 474,098 samples
  Memory usage: 1.73 GB

  Total samples: 474,098
  Total compounds: 10,713
  Cell lines: 141
  Gene features: 978
  Samples per compound (mean): 44.3
  Samples per compound (median): 36
  Compounds with >100 observations: 662

📊 Comparison with paper (SI page 2):